<a href="https://colab.research.google.com/github/juhumkwon/source_code/blob/main/Faster_R_CNN(%ED%95%99%EC%8A%B5%EB%AA%A8%EB%8D%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np

# ============================================================
# 1. Backbone 정의 (예: ResNet50, 최상위 FC layer 제외)
# ============================================================
backbone = tf.keras.applications.ResNet50(include_top=False, input_shape=(224, 224, 3))
# backbone.output 의 shape는 (batch, 7, 7, 2048) (입력 크기에 따라 달라짐)
feature_map = backbone.output

# ============================================================
# 2. RPN (Region Proposal Network) 헤드 정의
#    - 3x3 Convolution으로 특징을 추출한 후,
#      각 앵커(anchor)마다 객체 존재 여부와 박스 회귀 값을 예측
# ============================================================
def rpn_head(feature_map, num_anchors):
    x = layers.Conv2D(512, (3, 3), padding="same", activation='relu')(feature_map)
    # 객체 존재 확률 (각 위치에서 num_anchors 개의 score)
    rpn_class = layers.Conv2D(num_anchors, (1, 1), activation='sigmoid')(x)
    # 박스 회귀: 각 앵커당 4개 값 (x, y, w, h)
    rpn_bbox = layers.Conv2D(num_anchors * 4, (1, 1))(x)
    return rpn_class, rpn_bbox

num_anchors = 9  # 예시로 9개의 앵커 사용
rpn_class, rpn_bbox = rpn_head(feature_map, num_anchors)

# 이름을 부여하여 나중에 손실에 반영
rpn_class = layers.Activation('sigmoid', name='rpn_class_output')(rpn_class)
rpn_bbox  = layers.Lambda(lambda x: x, name='rpn_bbox_output')(rpn_bbox)

# ============================================================
# 3. ROI Pooling 및 최종 검출 헤드 (매우 단순화)
#    - 실제 Faster R-CNN은 RPN에서 생성된 후보 영역(ROI)에 대해
#      ROI Pooling/Align을 수행하지만, 여기서는 전체 feature map에 대해
#      GlobalAveragePooling을 적용하여 단일 벡터를 얻는 방식으로 대체합니다.
# ============================================================
roi_pool = layers.GlobalAveragePooling2D()(feature_map)
fc = layers.Dense(256, activation='relu')(roi_pool)

# 최종 분류 헤드 (예시: 두 클래스, object vs background)
cls_output = layers.Dense(2, activation='softmax', name='cls_output')(fc)
# 최종 박스 회귀 헤드 (4개 좌표)
bbox_output = layers.Dense(4, name='bbox_output')(fc)

# ============================================================
# 4. Faster R-CNN (단순화 버전) 모델 생성
#    - 입력: 이미지
#    - 출력: RPN의 예측(객체 존재 score 및 박스 회귀)와
#           최종 검출 헤드의 클래스, 박스 회귀 예측
# ============================================================
model = Model(inputs=backbone.input,
              outputs=[rpn_class, rpn_bbox, cls_output, bbox_output])

# ============================================================
# 5. 모델 컴파일
#    - 여기서는 RPN 관련 출력은 학습 손실에 반영하지 않고(가중치 0),
#      최종 검출 헤드(클래스, 박스 회귀)만 학습하도록 설정합니다.
# ============================================================
model.compile(optimizer='adam',
              loss={
                  "rpn_class_output": "binary_crossentropy",
                  "rpn_bbox_output": "mean_squared_error",
                  "cls_output": "categorical_crossentropy",
                  "bbox_output": "mean_squared_error"
              },
              loss_weights={
                  "rpn_class_output": 0.0,  # RPN 손실은 무시 (단순화)
                  "rpn_bbox_output": 0.0,
                  "cls_output": 1.0,
                  "bbox_output": 1.0
              },
              metrics={"cls_output": "accuracy"})

model.summary()

# ============================================================
# 6. 더미 데이터 생성 및 학습
#    - 실제 Faster R-CNN 학습에는 이미지, 앵커, ROI, 정답 박스 등이 필요하지만,
#      여기서는 개념 증명을 위한 더미 데이터를 사용합니다.
# ============================================================
batch_size = 2
dummy_images = np.random.rand(batch_size, 224, 224, 3).astype(np.float32)

# RPN 출력 더미 데이터 (shape는 backbone feature map 크기에 맞춤)
dummy_rpn_class_out = np.random.rand(batch_size, 7, 7, num_anchors).astype(np.float32)
dummy_rpn_bbox_out  = np.random.rand(batch_size, 7, 7, num_anchors * 4).astype(np.float32)

# 최종 검출 헤드 더미 데이터
# 분류: 2 클래스 (예: [1, 0]은 background, [0, 1]은 object)
dummy_cls = np.random.randint(0, 2, size=(batch_size,))
dummy_cls = tf.keras.utils.to_categorical(dummy_cls, num_classes=2)
# 박스 회귀: (x, y, w, h) 좌표 (정규화된 값이라고 가정)
dummy_bbox = np.random.rand(batch_size, 4).astype(np.float32)

dummy_outputs = {
    "rpn_class_output": dummy_rpn_class_out,
    "rpn_bbox_output": dummy_rpn_bbox_out,
    "cls_output": dummy_cls,
    "bbox_output": dummy_bbox
}

# 모델을 더미 데이터로 3 에포크 학습
model.fit(dummy_images, dummy_outputs, epochs=3, batch_size=batch_size)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 33,574,579 (128.08 MB)

 Trainable params: 33,521,459 (127.87 MB)

 Non-trainable params: 53,120 (207.50 KB)

Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 71s 71s/step - bbox_output_loss: 0.1674 - cls_output_accuracy: 1.0000 - cls_output_loss: 0.5997 - loss: 0.7671 - rpn_bbox_output_loss: 2.2192 - rpn_class_output_loss: 0.7196
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - bbox_output_loss: 5.0279 - cls_output_accuracy: 1.0000 - cls_output_loss: 0.0189 - loss: 5.0468 - rpn_bbox_output_loss: 2.7092 - rpn_class_output_loss: 0.7275
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step - bbox_output_loss: 2.3691 - cls_output_accuracy: 1.0000 - cls_output_loss: 0.2029 - loss: 2.5720 - rpn_bbox_output_loss: 2.2730 - rpn_class_output_loss: 0.7212
